# Clustering

#### Imports and Functions

In [1]:
import json
import cv2
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
from tqdm import tqdm
import psutil

#weighted distance density
def calculate_density(centers, k, p, epsilon):
    """
    Calculate density with inverse distance weighting (higher influence on larger distances).
    p: The power of the inverse distance. Higher values of p give even more weight to larger distances.
    epsilon: Small value to prevent division by zero for very close points.
    """
    nbrs = NearestNeighbors(n_neighbors=k).fit(centers)
    distances, _ = nbrs.kneighbors(centers)
    
    # Add epsilon to distances to avoid division by zero
    distances = distances[:, 1:] + epsilon  # Ignore the point itself, use distances[:, 1:]
    
    # Apply inverse distance weighting (1/distance^p)
    weights = (distances ** p)
    weighted_density_scores = np.mean(weights, axis=1)

    
    return weighted_density_scores


# Perform the KNN and density filtering for the specified number of iterations
def knn(k, contour_centers, method, iterations, p, epsilon):
    for iteration in tqdm(range(iterations), desc=f"Iterations for {method}"):
        # Calculate density using KNN
        density_scores_k = calculate_density(contour_centers, k, p, epsilon)

        # Apply a logarithmic transformation to the density scores
        #epsilon = 1e-5  # Small value to avoid log(0)
        log_density_scores_k = np.log(density_scores_k + epsilon)

        # Filter out points with a log density score above the threshold
        filtered_indices_d = log_density_scores_k <= d
        contour_centers = contour_centers[filtered_indices_d]
        log_density_scores_k = log_density_scores_k[filtered_indices_d]
        #k += 1
        print(len(contour_centers))

        # Check if there are no points left (all points filtered out)
        if len(contour_centers) == 0:
            print("No points remaining after filtering.")
            break
    return (contour_centers, log_density_scores_k, filtered_indices_d)

# Path to your images
image_folder = "D:\\PAIND\\DATA\\20240527_Data_Prozess_01\\webcam\\batch"
images = sorted([img for img in os.listdir(image_folder) if img.endswith(".jpg")])
k = 20  # Single value for k
d = 0.005  # Single density threshold
iterations = 10  # Number of iterations to loop through the KNN and density filtering
num_images_to_analyze = len(images)

# Initialize background subtractor
fgbg = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=50, detectShadows=True)

# List to store the centers of contours
contour_centers = []

# Base output folder path (same path as the image folder)
base_output_path = image_folder

# Update output folders to be within the same path as the image folder
output_folders = {
    "grayscale_gaussian": os.path.join(base_output_path, "output_grayscale_gaussian"),
    "hsv_value": os.path.join(base_output_path, "output_hsv_value"),
    "canny_edges": os.path.join(base_output_path, "output_canny_edges"),
    "mog2_background_subtraction": os.path.join(base_output_path, "output_mog2_background"),
    "farneback_optical_flow": os.path.join(base_output_path, "output_optical_flow"),
}

# Create output folders if they don't exist
for folder in output_folders.values():
    if not os.path.exists(folder):
        os.makedirs(folder, exist_ok=True)


#### Image Analysis

- Each method is applied to the images
- contours centres are saved to json files for later analysis

##### Grayscale Gaussian Blur

In [ ]:
# Initialize a dictionary to hold the results for Grayscale Gaussian approach
results = {"grayscale_gaussian": []}

# Monitor RAM usage
process = psutil.Process()

# Initialize the tqdm progress bars
progress_bar = tqdm(range(min(len(images) - 1, num_images_to_analyze)), desc="Processing Grayscale Gaussian", position=0)
ram_bar = tqdm(total=0, bar_format="{desc}", position=1)  # Second tqdm line for RAM usage

# Process the images for the Grayscale Gaussian approach
for i in progress_bar:
    img1_path = os.path.join(image_folder, images[i])
    img2_path = os.path.join(image_folder, images[i + 1])
    img1 = cv2.imread(img1_path)
    img2 = cv2.imread(img2_path)
    image_name = os.path.basename(img1_path)

    # Grayscale Gaussian approach
    gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    gray1_blurred = cv2.GaussianBlur(gray1, (21, 21), 0)
    gray2_blurred = cv2.GaussianBlur(gray2, (21, 21), 0)
    diff_gray = cv2.absdiff(gray1_blurred, gray2_blurred)
    _, thresh_gray = cv2.threshold(diff_gray, 25, 255, cv2.THRESH_BINARY)
    contours_gray, _ = cv2.findContours(thresh_gray.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contour_centers = [(x + w // 2, y + h // 2) for (x, y, w, h) in [cv2.boundingRect(c) for c in contours_gray]]
    results["grayscale_gaussian"].append({"image_name": image_name, "contour_centers": contour_centers})

    # Update RAM usage bar
    ram_usage_mb = process.memory_info().rss / (1024 * 1024)  # Convert bytes to MB
    ram_bar.set_description(f"RAM Usage: {ram_usage_mb:.2f} MB")

# Save results for Grayscale Gaussian
json_filename = os.path.join(output_folders["grayscale_gaussian"], "grayscale_gaussian_contour_centers.json")
with open(json_filename, 'w') as json_file:
    json.dump(results["grayscale_gaussian"], json_file, indent=4)

print("Grayscale Gaussian results saved.")


In [ ]:
print(ram_usage_mb)

##### Hsv Value

In [ ]:
# Initialize a dictionary to hold the results for HSV Value approach
results = {"hsv_value": []}

# Monitor RAM usage
process = psutil.Process()

# Initialize the tqdm progress bars
progress_bar = tqdm(range(min(len(images) - 1, num_images_to_analyze)), desc="Processing hsv", position=0)
ram_bar = tqdm(total=0, bar_format="{desc}", position=1)  # Second tqdm line for RAM usage

# Process the images for the HSV Value approach
for i in tqdm(range(min(len(images) - 1, num_images_to_analyze)), desc="Processing HSV Value"):
    img1_path = os.path.join(image_folder, images[i])
    img2_path = os.path.join(image_folder, images[i + 1])
    img1 = cv2.imread(img1_path)
    img2 = cv2.imread(img2_path)
    image_name = os.path.basename(img1_path)

    # HSV Value channel approach
    hsv1 = cv2.cvtColor(img1, cv2.COLOR_BGR2HSV)
    hsv2 = cv2.cvtColor(img2, cv2.COLOR_BGR2HSV)
    diff_hsv = cv2.absdiff(hsv1[:, :, 2], hsv2[:, :, 2])
    _, thresh_hsv = cv2.threshold(diff_hsv, 25, 255, cv2.THRESH_BINARY)
    contours_hsv, _ = cv2.findContours(thresh_hsv.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contour_centers = [(x + w // 2, y + h // 2) for (x, y, w, h) in [cv2.boundingRect(c) for c in contours_hsv]]
    results["hsv_value"].append({"image_name": image_name, "contour_centers": contour_centers})

    # Update RAM usage bar
    ram_usage_mb = process.memory_info().rss / (1024 * 1024)  # Convert bytes to MB
    ram_bar.set_description(f"RAM Usage: {ram_usage_mb:.2f} MB")

# Save results for HSV Value
json_filename = os.path.join(output_folders["hsv_value"], "hsv_value_contour_centers.json")
with open(json_filename, 'w') as json_file:
    json.dump(results["hsv_value"], json_file, indent=4)
print("HSV Value results saved.")


##### Canny Edge
not used: too much memory usage at knn

In [ ]:
"""# Initialize a dictionary to hold the results for Canny Edge approach
results = {"canny_edges": []}

# Monitor RAM usage
process = psutil.Process()

# Initialize the tqdm progress bars
progress_bar = tqdm(range(min(len(images) - 1, num_images_to_analyze)), desc="Processing Canny Edges", position=0)
ram_bar = tqdm(total=0, bar_format="{desc}", position=1)  # Second tqdm line for RAM usage

# Process the images for the Canny Edge approach
for i in tqdm(range(min(len(images) - 1, num_images_to_analyze)), desc="Processing Canny Edges"):
    img1_path = os.path.join(image_folder, images[i])
    img2_path = os.path.join(image_folder, images[i + 1])
    img1 = cv2.imread(img1_path)
    img2 = cv2.imread(img2_path)
    image_name = os.path.basename(img1_path)

    # Canny edge detection approach
    edges1 = cv2.Canny(img1, 50, 150)
    edges2 = cv2.Canny(img2, 50, 150)
    diff_edges = cv2.absdiff(edges1, edges2)
    _, thresh_edges = cv2.threshold(diff_edges, 25, 255, cv2.THRESH_BINARY)
    contours_edges, _ = cv2.findContours(thresh_edges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contour_centers = [(x + w // 2, y + h // 2) for (x, y, w, h) in [cv2.boundingRect(c) for c in contours_edges]]
    results["canny_edges"].append({"image_name": image_name, "contour_centers": contour_centers})

    # Update RAM usage bar
    ram_usage_mb = process.memory_info().rss / (1024 * 1024)  # Convert bytes to MB
    ram_bar.set_description(f"RAM Usage: {ram_usage_mb:.2f} MB")

# Save results for Canny Edge
json_filename = os.path.join(output_folders["canny_edges"], "canny_edges_contour_centers.json")
with open(json_filename, 'w') as json_file:
    json.dump(results["canny_edges"], json_file, indent=4)
print("Canny Edges results saved.")
"""

##### Mog2 Background Subtraction
not used: too much memory usage at knn

In [ ]:
"""# Initialize a dictionary to hold the results for MOG2 background subtraction approach
results = {"mog2_background_subtraction": []}

# Monitor RAM usage
process = psutil.Process()

# Initialize the tqdm progress bars
progress_bar = tqdm(range(min(len(images) - 1, num_images_to_analyze)), desc="Processing Mog2", position=0)
ram_bar = tqdm(total=0, bar_format="{desc}", position=1)  # Second tqdm line for RAM usage

# Process the images for the MOG2 background subtraction approach
for i in tqdm(range(min(len(images) - 1, num_images_to_analyze)), desc="Processing MOG2 Background Subtraction"):
    img1_path = os.path.join(image_folder, images[i])
    img2_path = os.path.join(image_folder, images[i + 1])
    img1 = cv2.imread(img1_path)
    img2 = cv2.imread(img2_path)
    image_name = os.path.basename(img1_path)

    # MOG2 background subtraction approach
    fgmask1 = fgbg.apply(img1)
    fgmask2 = fgbg.apply(img2)
    diff_mog2 = cv2.absdiff(fgmask1, fgmask2)
    _, thresh_mog2 = cv2.threshold(diff_mog2, 25, 255, cv2.THRESH_BINARY)
    contours_mog2, _ = cv2.findContours(thresh_mog2.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contour_centers = [(x + w // 2, y + h // 2) for (x, y, w, h) in [cv2.boundingRect(c) for c in contours_mog2]]
    results["mog2_background_subtraction"].append({"image_name": image_name, "contour_centers": contour_centers})

    # Update RAM usage bar
    ram_usage_mb = process.memory_info().rss / (1024 * 1024)  # Convert bytes to MB
    ram_bar.set_description(f"RAM Usage: {ram_usage_mb:.2f} MB")

# Save results for MOG2 background subtraction
json_filename = os.path.join(output_folders["mog2_background_subtraction"], "mog2_background_subtraction_contour_centers.json")
with open(json_filename, 'w') as json_file:
    json.dump(results["mog2_background_subtraction"], json_file, indent=4)
print("MOG2 Background Subtraction results saved.")
"""

##### Farneback Optical Flow

In [ ]:
# Initialize a dictionary to hold the results for Farneback Optical Flow approach
results = {"farneback_optical_flow": []}

# Monitor RAM usage
process = psutil.Process()

# Initialize the tqdm progress bars
progress_bar = tqdm(range(min(len(images) - 1, num_images_to_analyze)), desc="Processing Farneback", position=0)
ram_bar = tqdm(total=0, bar_format="{desc}", position=1)  # Second tqdm line for RAM usage

# Process the images for the Farneback Optical Flow approach
for i in tqdm(range(min(len(images) - 1, num_images_to_analyze)), desc="Processing Farneback Optical Flow"):
    img1_path = os.path.join(image_folder, images[i])
    img2_path = os.path.join(image_folder, images[i + 1])
    img1 = cv2.imread(img1_path)
    img2 = cv2.imread(img2_path)
    image_name = os.path.basename(img1_path)

    # Convert images to grayscale
    gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    # Farneback optical flow approach
    flow = cv2.calcOpticalFlowFarneback(gray1, gray2, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    diff_optical_flow = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
    _, thresh_optical_flow = cv2.threshold(diff_optical_flow, 25, 255, cv2.THRESH_BINARY)
    contours_optical_flow, _ = cv2.findContours(thresh_optical_flow.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contour_centers = [(x + w // 2, y + h // 2) for (x, y, w, h) in [cv2.boundingRect(c) for c in contours_optical_flow]]
    results["farneback_optical_flow"].append({"image_name": image_name, "contour_centers": contour_centers})

    # Log RAM usage dynamically
    ram_usage_mb = process.memory_info().rss / (1024 * 1024)  # Convert bytes to MB
    progress_bar.set_description(f"Processing {image_name} | RAM Usage: {ram_usage_mb:.2f} MB")

# Save results for Farneback Optical Flow
json_filename = os.path.join(output_folders["farneback_optical_flow"], "farneback_optical_flow_contour_centers.json")
with open(json_filename, 'w') as json_file:
    json.dump(results["farneback_optical_flow"], json_file, indent=4)
print("Farneback Optical Flow results saved.")


#### check number of contours for each method

In [ ]:
# Output the number of contour centers in each method's JSON file
for method, folder in output_folders.items():
    json_filename = os.path.join(folder, f"{method}_contour_centers.json")
    
    # Load the contour centers from the JSON file
    with open(json_filename, 'r') as json_file:
        data = json.load(json_file)

    # Count the total number of contour centers across all images
    total_contour_centers = sum(len(item["contour_centers"]) for item in data)
    
    # Output the results
    print(f"MethoE: {method}, Number of Contour Centers: {total_contour_centers}")


### Reduction of contours

- load json 
- reduce contours using knn
- plot reduced centers

#### Parameter Values for KNN

In [4]:
k = 20  # Single value for k
d = 0.05  # Single density threshold
p = 20
epsilon = 1e-7
iterations = 5  # Number of iterations to loop through the KNN and density filtering

In [ ]:
for method, folder in tqdm(output_folders.items(), desc="Processing Methods"):
    json_filename = os.path.join(folder, f"{method}_contour_centers.json")
    print(json_filename)
    print(method)
    print(folder)
    print("")

#### Grayscale Gaussian
ttr = 11m 41.4s

In [ ]:
method = "grayscale_gaussian"
json_filename = r"D:\PAIND\DATA\20240527_Data_Prozess_01\webcam\batch\output_grayscale_gaussian\grayscale_gaussian_contour_centers.json"
folder = r"D:\PAIND\DATA\20240527_Data_Prozess_01\webcam\batch\output_grayscale_gaussian"



# Load the contour centers from the JSON file
with open(json_filename, 'r') as json_file:
    data = json.load(json_file)

# Extract the contour centers for all images
contour_centers = []
for item in tqdm(data, desc=f"Extracting Contours for {method}"):
    contour_centers.extend(item["contour_centers"])

contour_centers = np.array(contour_centers)

contour_centers_grayscale, log_density_scores_k_grayscale, filtered_indices_d = knn(k, contour_centers, method, iterations, p, epsilon)

# Normalize the final filtered log density scores for color mapping
norm_log_density_scores_final_grayscale = (log_density_scores_k_grayscale - log_density_scores_k_grayscale.min()) / (log_density_scores_k_grayscale.max() - log_density_scores_k_grayscale.min())

# Create a color map based on log density (red for high density, blue for low density)
colors_final_grayscale = plt.cm.jet(norm_log_density_scores_final_grayscale)

# Plot the final result
fig, ax = plt.subplots(figsize=(10, 8))
scatter_final = ax.scatter(contour_centers_grayscale[:, 0], contour_centers_grayscale[:, 1], color=colors_final_grayscale, s=10)
ax.set_title(f"{method} - Contour Centers with Log Density after {iterations} Iterations\n(k={k-iterations}-{k}, d={d})", fontsize=10)
ax.set_xlabel("X Coordinate")
ax.set_ylabel("Y Coordinate")
ax.invert_yaxis()
plt.colorbar(scatter_final, ax=ax, label='Log Density (neighbor distance)')

plt.tight_layout()

# Save the plot to the same location as the JSON file
plot_filename = os.path.join(folder, f"{method}_contour_centers_plot.png")
plt.savefig(plot_filename)
plt.close(fig)

print(f"Plot saved to {plot_filename}")


#### Hsv
ttr = 395min 42.9s

In [ ]:
method = "hsv_value"
json_filename = r"D:\PAIND\DATA\20240527_Data_Prozess_01\webcam\batch\output_hsv_value\hsv_value_contour_centers.json"
folder = r"D:\PAIND\DATA\20240527_Data_Prozess_01\webcam\batch\output_hsv_value"


# Load the contour centers from the JSON file
with open(json_filename, 'r') as json_file:
    data = json.load(json_file)

# Extract the contour centers for all images
contour_centers = []
for item in tqdm(data, desc=f"Extracting Contours for {method}"):
    contour_centers.extend(item["contour_centers"])

contour_centers = np.array(contour_centers)


contour_centers_hsv, log_density_scores_k_hsv, filtered_indices_d = knn(k, contour_centers, method, iterations, p, epsilon)


# Normalize the final filtered log density scores for color mapping
norm_log_density_scores_final_hsv = (log_density_scores_k_hsv - log_density_scores_k_hsv.min()) / (log_density_scores_k_hsv.max() - log_density_scores_k_hsv.min())

# Create a color map based on log density (red for high density, blue for low density)
colors_final_hsv = plt.cm.jet(norm_log_density_scores_final_hsv)

# Plot the final result
fig, ax = plt.subplots(figsize=(10, 8))
scatter_final = ax.scatter(contour_centers_hsv[:, 0], contour_centers_hsv[:, 1], color=colors_final_hsv, s=10)
ax.set_title(f"{method} - Contour Centers with Log Density after {iterations} Iterations\n(k={k-iterations}-{k}, d={d})", fontsize=10)
ax.set_xlabel("X Coordinate")
ax.set_ylabel("Y Coordinate")
ax.invert_yaxis()
plt.colorbar(scatter_final, ax=ax, label='Log Density (neighbor distance)')

plt.tight_layout()

# Save the plot to the same location as the JSON file
plot_filename = os.path.join(folder, f"{method}_contour_centers_plot.png")
plt.savefig(plot_filename)
plt.close(fig)

print(f"Plot saved to {plot_filename}")


#### Canny Edge
not enough memory

In [ ]:
"""method = "canny_edges"
json_filename = r"D:\PAIND\DATA\20240527_Data_Prozess_01\webcam\batch\output_canny_edges\canny_edges_contour_centers.json"
folder = r"D:\PAIND\DATA\20240527_Data_Prozess_01\webcam\batch\output_canny_edges"



# Load the contour centers from the JSON file
with open(json_filename, 'r') as json_file:
    data = json.load(json_file)

# Extract the contour centers for all images
contour_centers = []
for item in tqdm(data, desc=f"Extracting Contours for {method}"):
    contour_centers.extend(item["contour_centers"])

contour_centers = np.array(contour_centers)

# Perform the KNN and density filtering for the specified number of iterations
for iteration in tqdm(range(iterations), desc=f"Iterations for {method}"):
    # Calculate density using KNN
    density_scores_k = calculate_density(contour_centers, k)

    # Apply a logarithmic transformation to the density scores
    epsilon = 1e-5  # Small value to avoid log(0)
    log_density_scores_k = np.log(density_scores_k + epsilon)

    # Filter out points with a log density score above the threshold
    filtered_indices_d = log_density_scores_k <= d
    contour_centers = contour_centers[filtered_indices_d]
    log_density_scores_k = log_density_scores_k[filtered_indices_d]
    #k += 1
    print(len(contour_centers))

    # Check if there are no points left (all points filtered out)
    if len(contour_centers) == 0:
        print("No points remaining after filtering.")
        break

# Normalize the final filtered log density scores for color mapping
norm_log_density_scores_final = (log_density_scores_k - log_density_scores_k.min()) / (log_density_scores_k.max() - log_density_scores_k.min())

# Create a color map based on log density (red for high density, blue for low density)
colors_final = plt.cm.jet(norm_log_density_scores_final)

# Plot the final result
fig, ax = plt.subplots(figsize=(10, 8))
scatter_final = ax.scatter(contour_centers[:, 0], contour_centers[:, 1], color=colors_final, s=10)
ax.set_title(f"{method} - Contour Centers with Log Density after {iterations} Iterations\n(k={k-iterations}-{k}, d={d})", fontsize=10)
ax.set_xlabel("X Coordinate")
ax.set_ylabel("Y Coordinate")
ax.invert_yaxis()
plt.colorbar(scatter_final, ax=ax, label='Log Density (neighbor distance)')

plt.tight_layout()

# Save the plot to the same location as the JSON file
plot_filename = os.path.join(folder, f"{method}_contour_centers_plot.png")
plt.savefig(plot_filename)
plt.close(fig)

print(f"Plot saved to {plot_filename}")
"""

#### Mog2
not enough memory

In [ ]:
"""method = "mog2_background_subtraction"
json_filename = r"D:\PAIND\DATA\20240527_Data_Prozess_01\webcam\batch\output_mog2_background\mog2_background_subtraction_contour_centers.json"
folder = r"D:\PAIND\DATA\20240527_Data_Prozess_01\webcam\batch\output_mog2_background"



# Load the contour centers from the JSON file
with open(json_filename, 'r') as json_file:
    data = json.load(json_file)

# Extract the contour centers for all images
contour_centers = []
for item in tqdm(data, desc=f"Extracting Contours for {method}"):
    contour_centers.extend(item["contour_centers"])

contour_centers = np.array(contour_centers)

# Perform the KNN and density filtering for the specified number of iterations
for iteration in tqdm(range(iterations), desc=f"Iterations for {method}"):
    # Calculate density using KNN
    density_scores_k = calculate_density(contour_centers, k)

    # Apply a logarithmic transformation to the density scores
    epsilon = 1e-5  # Small value to avoid log(0)
    log_density_scores_k = np.log(density_scores_k + epsilon)

    # Filter out points with a log density score above the threshold
    filtered_indices_d = log_density_scores_k <= d
    contour_centers = contour_centers[filtered_indices_d]
    log_density_scores_k = log_density_scores_k[filtered_indices_d]
    #k += 1
    print(len(contour_centers))

    # Check if there are no points left (all points filtered out)
    if len(contour_centers) == 0:
        print("No points remaining after filtering.")
        break

# Normalize the final filtered log density scores for color mapping
norm_log_density_scores_final = (log_density_scores_k - log_density_scores_k.min()) / (log_density_scores_k.max() - log_density_scores_k.min())

# Create a color map based on log density (red for high density, blue for low density)
colors_final = plt.cm.jet(norm_log_density_scores_final)

# Plot the final result
fig, ax = plt.subplots(figsize=(10, 8))
scatter_final = ax.scatter(contour_centers[:, 0], contour_centers[:, 1], color=colors_final, s=10)
ax.set_title(f"{method} - Contour Centers with Log Density after {iterations} Iterations\n(k={k-iterations}-{k}, d={d})", fontsize=10)
ax.set_xlabel("X Coordinate")
ax.set_ylabel("Y Coordinate")
ax.invert_yaxis()
plt.colorbar(scatter_final, ax=ax, label='Log Density (neighbor distance)')

plt.tight_layout()

# Save the plot to the same location as the JSON file
plot_filename = os.path.join(folder, f"{method}_contour_centers_plot.png")
plt.savefig(plot_filename)
plt.close(fig)

print(f"Plot saved to {plot_filename}")
"""

#### Farneback
ttr = 150min 48s

In [ ]:
method = "farneback_optical_flow"
json_filename = r"D:\PAIND\DATA\20240527_Data_Prozess_01\webcam\batch\output_optical_flow\farneback_optical_flow_contour_centers.json"
folder = r"D:\PAIND\DATA\20240527_Data_Prozess_01\webcam\batch\output_optical_flow"



# Load the contour centers from the JSON file
with open(json_filename, 'r') as json_file:
    data = json.load(json_file)

# Extract the contour centers for all images
contour_centers = []
for item in tqdm(data, desc=f"Extracting Contours for {method}"):
    contour_centers.extend(item["contour_centers"])

contour_centers = np.array(contour_centers)


contour_centers_farneback, log_density_scores_k_farneback, filtered_indices_d = knn(k, contour_centers, method, iterations, p, epsilon)

# Normalize the final filtered log density scores for color mapping
norm_log_density_scores_final_farneback = (log_density_scores_k_farneback - log_density_scores_k_farneback.min()) / (log_density_scores_k_farneback.max() - log_density_scores_k_farneback.min())

# Create a color map based on log density (red for high density, blue for low density)
colors_final_farneback = plt.cm.jet(norm_log_density_scores_final_farneback)

# Plot the final result
fig, ax = plt.subplots(figsize=(10, 8))
scatter_final = ax.scatter(contour_centers_farneback[:, 0], contour_centers_farneback[:, 1], color=colors_final_farneback, s=10)
ax.set_title(f"{method} - Contour Centers with Log Density after {iterations} Iterations\n(k={k-iterations}-{k}, d={d})", fontsize=10)
ax.set_xlabel("X Coordinate")
ax.set_ylabel("Y Coordinate")
ax.invert_yaxis()
plt.colorbar(scatter_final, ax=ax, label='Log Density (neighbor distance)')

plt.tight_layout()

# Save the plot to the same location as the JSON file
plot_filename = os.path.join(folder, f"{method}_contour_centers_plot.png")
plt.savefig(plot_filename)
plt.close(fig)

print(f"Plot saved to {plot_filename}")


#### Save KNNs to disk

In [ ]:
# Base path to the output folders
base_output_path = r"D:\PAIND\DATA\20240527_Data_Prozess_01\webcam\batch"

# Dynamically determine the folder paths and methods
output_folders = {
    "grayscale_gaussian": os.path.join(base_output_path, "output_grayscale_gaussian"),
    "hsv_value": os.path.join(base_output_path, "output_hsv_value"),
    "canny_edges": os.path.join(base_output_path, "output_canny_edges"),
    "mog2_background_subtraction": os.path.join(base_output_path, "output_mog2_background"),
    "farneback_optical_flow": os.path.join(base_output_path, "output_optical_flow"),
}

# Dictionary to map methods to their corresponding contour centers
knns = {
    "grayscale_gaussian": contour_centers_grayscale,
    "farneback_optical_flow": contour_centers_farneback,
    "hsv_value": contour_centers_hsv,
    # Add more methods and their corresponding data if available
}

# Iterate through methods and save results
for method, contour_centers in tqdm(knns.items(), desc="Saving KNN Results"):
    # Dynamically locate the folder for the method
    folder = output_folders.get(method)
    if folder is None:
        print(f"No folder found for method: {method}. Skipping...")
        continue

    # Generate the JSON filename dynamically based on the method
    json_filename = os.path.join(folder, f"{method}_knn_contour_centers.json")

    # Prepare the data to save
    results = [{"contour_centers": contour.tolist()} for contour in contour_centers]

    # Ensure the folder exists
    os.makedirs(folder, exist_ok=True)

    # Save the results as a JSON file
    with open(json_filename, 'w') as json_file:
        json.dump(results, json_file, indent=4)

    print(f"KNN results for {method} saved to {json_filename}")

### Mask Creation

In [ ]:
method = "grayscale_gaussian"
json_filename = r"D:\PAIND\DATA\20240527_Data_Prozess_01\webcam\batch\output_grayscale_gaussian\grayscale_gaussian_knn_contour_centers.json"
folder = r"D:\PAIND\DATA\20240527_Data_Prozess_01\webcam\batch\output_grayscale_gaussian"

# Load the contour centers from the JSON file
with open(json_filename, 'r') as json_file:
    data = json.load(json_file)

contour_centers = []
for item in tqdm(data, desc=f"Extracting Contours for {method}"):
    contour_centers.extend(item["contour_centers"])

contour_centers = np.array(contour_centers)

# Mask Creation

In [ ]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt

# Define paths and parameters
example_img_path = "D:\\PAIND\\DATA\\20240527_Data_Prozess_01\\webcam\\batch\\20240527_140517546_imgwebcam.jpg"
contour_centers = np.array([[x, y] for x, y in zip(range(50, 500, 10), range(50, 500, 10))])  # Example data; replace with actual contour centers

# Load an example image to get the dimensions
example_img = cv2.imread(example_img_path)
if example_img is None:
    raise FileNotFoundError(f"Image at {example_img_path} could not be loaded.")

img_height, img_width = example_img.shape[:2]
y_threshold = 0  # Adjust this value based on your specific data and reel position

# Filter out contour centers above the Y-coordinate threshold (optional)
# Uncomment and adjust the line below if needed
# filtered_contour_centers = contour_centers[contour_centers[:, 1] > y_threshold]
filtered_contour_centers = contour_centers  # Use unfiltered centers if no filter is applied

# Create a convex hull of the filtered points
points = np.array(filtered_contour_centers, dtype=np.int32)
hull = cv2.convexHull(points)

# Draw the convex hull on a blank mask
mask = np.zeros((img_height, img_width), dtype=np.uint8)
cv2.drawContours(mask, [hull], -1, 255, -1)  # Fill the convex hull with white (255)

# Save the mask
output_mask_path = "filtered_convex_hull_mask.png"
cv2.imwrite(output_mask_path, mask)
print(f"Mask saved to {output_mask_path}")

# Display the mask
plt.figure(figsize=(10, 8))
plt.imshow(mask, cmap='gray')
plt.title("Filtered Convex Hull Mask")
plt.axis('off')
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Plot the mask on the left and the contour centers on the right
fig, axs = plt.subplots(1, 2, figsize=(15, 8))

# Left plot: Filtered convex hull mask with axis labels and matching extent
axs[0].imshow(mask, cmap='gray', extent=[0, img_width, img_height, 0])  # Set extent to match the image dimensions
axs[0].set_title("Filtered Convex Hull Mask")
axs[0].set_xlabel("X Coordinate (Width)")
axs[0].set_ylabel("Y Coordinate (Height)")
axs[0].axis('on')  # Ensure the axes are visible

# Right plot: Contour centers with axis labels
axs[1].scatter(filtered_contour_centers[:, 0], filtered_contour_centers[:, 1], c='red', s=10)
axs[1].invert_yaxis()  # Invert Y-axis to match image coordinates
axs[1].set_title("Filtered Contour Centers")
axs[1].set_xlabel("X Coordinate (Width)")
axs[1].set_ylabel("Y Coordinate (Height)")

plt.tight_layout()
plt.show()

#### troubleshooting contour centers

In [ ]:
import json
from pprint import pprint

# Define the file paths
flat_original_file = r"D:\PAIND\DATA\20240527_Data_Prozess_01\webcam\batch\output_grayscale_gaussian\flat_original_contour_centers.json"
flat_reduced_file = r"D:\PAIND\DATA\20240527_Data_Prozess_01\webcam\batch\output_grayscale_gaussian\flat_knn_contour_centers.json"
original_centers_file = r"D:\PAIND\DATA\20240527_Data_Prozess_01\webcam\batch\output_grayscale_gaussian\grayscale_gaussian_contour_centers.json"
knn_centers_file = r"D:\PAIND\DATA\20240527_Data_Prozess_01\webcam\batch\output_grayscale_gaussian\grayscale_gaussian_knn_contour_centers.json"

# Load the original centers
with open(flat_original_file, 'r') as f:
    original_centers = json.load(f)

# Load the KNN centers
with open(flat_reduced_file, 'r') as f:
    knn_centers = json.load(f)

# Print the first couple of lines for comparison
print("Original Contour Centers:")
pprint(original_centers[:10])

print("\nKNN Reduced Contour Centers:")
pprint(knn_centers[:10])


adjust json storage format

In [ ]:
import json
import numpy as np

# Paths to the JSON files
original_json_path = r"D:\PAIND\DATA\20240527_Data_Prozess_01\webcam\batch\output_grayscale_gaussian\grayscale_gaussian_contour_centers.json"
reduced_json_path = r"D:\PAIND\DATA\20240527_Data_Prozess_01\webcam\batch\output_grayscale_gaussian\grayscale_gaussian_knn_contour_centers.json"

# Load the original contour centers
with open(original_json_path, 'r') as f:
    original_data = json.load(f)

# Load the KNN-reduced contour centers
with open(reduced_json_path, 'r') as f:
    reduced_data = json.load(f)

# Flatten the original contour centers
flat_original_centers = []
for entry in original_data:
    flat_original_centers.extend(entry["contour_centers"])

flat_original_centers = np.array(flat_original_centers, dtype=np.int32)

# Convert the reduced contour centers into a flat structure if not already
flat_reduced_centers = np.array(
    [entry["contour_centers"] for entry in reduced_data], dtype=np.int32
)

# Save the flattened structures back to JSON
flat_original_file = r"D:\PAIND\DATA\20240527_Data_Prozess_01\webcam\batch\output_grayscale_gaussian\flat_original_contour_centers.json"
flat_reduced_file = r"D:\PAIND\DATA\20240527_Data_Prozess_01\webcam\batch\output_grayscale_gaussian\flat_knn_contour_centers.json"

with open(flat_original_file, 'w') as f:
    json.dump(flat_original_centers.tolist(), f, indent=4)

with open(flat_reduced_file, 'w') as f:
    json.dump(flat_reduced_centers.tolist(), f, indent=4)

print(f"Flattened original centers saved to {flat_original_file}")
print(f"Flattened reduced centers saved to {flat_reduced_file}")


#### switch contour centers here

In [ ]:
import cv2
import numpy as np
import os
import json
import matplotlib.pyplot as plt
from tqdm import tqdm

# Parameters and file paths
method = "grayscale_gaussian"
json_filename = r"D:\PAIND\DATA\20240527_Data_Prozess_01\webcam\batch\output_grayscale_gaussian\grayscale_gaussian_knn_contour_centers.json"
folder = r"D:\PAIND\DATA\20240527_Data_Prozess_01\webcam\batch\output_grayscale_gaussian"
example_img_path = r"D:\PAIND\DATA\20240527_Data_Prozess_01\webcam\batch\20240527_140517546_imgwebcam.jpg"

# Load the contour centers from the JSON file
with open(flat_reduced_file, 'r') as json_file:
    contour_centers = json.load(json_file)

# Convert the flat list into a NumPy array
contour_centers = np.array(contour_centers, dtype=np.int32)

# Load an example image to get the dimensions
example_img = cv2.imread(example_img_path)
img_height, img_width = example_img.shape[:2]

# Set a Y-coordinate threshold to filter points
y_threshold = 0  # Adjust this value based on your specific data and reel position

# Filter out contour centers above the Y-coordinate threshold
filtered_contour_centers = contour_centers[contour_centers[:, 1] > y_threshold]

# Create a blank mask
mask = np.zeros((img_height, img_width), dtype=np.uint8)

# Plot the filtered points onto the mask
for point in filtered_contour_centers:
    x, y = int(point[0]), int(point[1])
    mask[y, x] = 255  # Set points to white on the mask

# Dilation to connect nearby points
kernel_size = 5  # Adjust kernel size as needed
kernel = np.ones((kernel_size, kernel_size), np.uint8)
mask_dilated = cv2.dilate(mask, kernel, iterations=1)

# Find contours in the dilated mask
contours, _ = cv2.findContours(mask_dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Approximate each contour with more points to increase corners
approximated_contours = []
for contour in contours:
    if cv2.contourArea(contour) > 100:  # Filter small contours based on area
        epsilon = 0.005 * cv2.arcLength(contour, True)  # Adjust epsilon for more/fewer corners
        approx = cv2.approxPolyDP(contour, epsilon, True)
        approximated_contours.append(approx)

# Draw all approximated contours on a new mask
mask_approx = np.zeros((img_height, img_width), dtype=np.uint8)
cv2.drawContours(mask_approx, approximated_contours, -1, 255, -1)  # Fill contours with white

# Plot the original dilated mask and the approximated contour mask side by side
fig, axs = plt.subplots(1, 2, figsize=(15, 8))

# Left plot: Mask with original dilated points
axs[0].imshow(mask_dilated, cmap='gray', extent=[0, img_width, img_height, 0])
axs[0].set_title("Mask with Original Dilated Contour")
axs[0].set_xlabel("X Coordinate (Width)")
axs[0].set_ylabel("Y Coordinate (Height)")

# Right plot: Mask with approximated contours
axs[1].imshow(mask_approx, cmap='gray', extent=[0, img_width, img_height, 0])
axs[1].set_title("Mask with Approximated Contour")
axs[1].set_xlabel("X Coordinate (Width)")
axs[1].set_ylabel("Y Coordinate (Height)")

plt.tight_layout()
plt.show()


In [ ]:
# Load the same example image
example_img = cv2.imread(example_img_path)

# Ensure the mask is in the same shape and type as the image (e.g., 3 channels)
mask_3channel = cv2.merge([mask_approx] * 3)

# Apply the mask to the example image
masked_image = cv2.bitwise_and(example_img, mask_3channel)

# Display the masked image
plt.figure(figsize=(10, 8))
plt.imshow(cv2.cvtColor(masked_image, cv2.COLOR_BGR2RGB))  # Convert BGR to RGB for correct color display
plt.title("Image with Mask Applied")
plt.axis("off")
plt.show()

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

# Parameters and file paths
img_height, img_width = 720, 1280  # Set based on your image size if known
example_img_path = "D:\\PAIND\\DATA\\20240527_Data_Prozess_01\\webcam\\batch\\20240527_140517546_imgwebcam.jpg"

# Create a blank mask
mask_outside = np.zeros((img_height, img_width), dtype=np.uint8)

# Define points for the left triangular mask, extending vertically upward from the current points
triangle_left_points = np.array([[0, 0], [200, 0], [200, 300], [0, 700]], dtype=np.int32)

# Define points for the right triangular mask (mirrored configuration), extending vertically upward
triangle_right_points = np.array([[img_width, 0], [img_width - 200, 0], [img_width - 200, 300], [img_width, 700]], dtype=np.int32)

# Draw the left triangle on the mask
cv2.fillPoly(mask_outside, [triangle_left_points], 255)  # Fill with white (255) for visibility

# Draw the right triangle on the mask
cv2.fillPoly(mask_outside, [triangle_right_points], 255)  # Fill with white (255) for visibility

# Load the example image and apply the mask overlay
example_img = cv2.imread(example_img_path)
overlayed_img = example_img.copy()
overlayed_img[mask_outside == 255] = [0, 0, 255]  # Color the masked region in red for visualization

# Plot the standalone mask and the image with the mask overlay
fig, axs = plt.subplots(1, 2, figsize=(15, 8))

# Left plot: Mask with triangular regions
axs[0].imshow(mask_outside, cmap='gray', extent=[0, img_width, img_height, 0])
axs[0].set_title("Mask with Left and Right Vertical Triangular Regions")
axs[0].set_xlabel("X Coordinate (Width)")
axs[0].set_ylabel("Y Coordinate (Height)")

# Right plot: Image with mask overlay
axs[1].imshow(cv2.cvtColor(overlayed_img, cv2.COLOR_BGR2RGB))
axs[1].set_title("Example Image with Mask Overlay")
axs[1].set_xlabel("X Coordinate (Width)")
axs[1].set_ylabel("Y Coordinate (Height)")

plt.tight_layout()
plt.show()


In [ ]:
# Perform the deduction
deducted_mask_outside = cv2.bitwise_and(mask_outside, cv2.bitwise_not(mask_approx))

# Plot the masks side by side
fig, axs = plt.subplots(1, 3, figsize=(20, 8))

# Original `another_mask`
axs[0].imshow(mask_outside, cmap='gray', extent=[0, img_width, img_height, 0])
axs[0].set_title("Original Mask")
axs[0].set_xlabel("X Coordinate (Width)")
axs[0].set_ylabel("Y Coordinate (Height)")

# `mask_approx`
axs[1].imshow(mask_approx, cmap='gray', extent=[0, img_width, img_height, 0])
axs[1].set_title("Mask to Deduct (mask_approx)")
axs[1].set_xlabel("X Coordinate (Width)")
axs[1].set_ylabel("Y Coordinate (Height)")

# Resulting `deducted_mask`
axs[2].imshow(deducted_mask_outside, cmap='gray', extent=[0, img_width, img_height, 0])
axs[2].set_title("Resulting Mask After Deduction")
axs[2].set_xlabel("X Coordinate (Width)")
axs[2].set_ylabel("Y Coordinate (Height)")

plt.tight_layout()
plt.show()

# Save the resulting mask for later use or inspection
output_path = os.path.join(folder, "deducted_mask_outside.png")
cv2.imwrite(output_path, deducted_mask_outside)
print(f"Deducted mask saved to {output_path}")


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Load the example image
example_img = cv2.imread(example_img_path)

# Assume `filtered_contour_centers` contains the reduced contour points for the reel
points = np.array(filtered_contour_centers, dtype=np.int32)

# Calculate the center X-coordinate of the image
center_x = img_width // 2

# Find the innermost points on the left and right closest to the center
left_point = min(points[points[:, 0] < center_x], key=lambda p: abs(p[0] - center_x))
right_point = min(points[points[:, 0] > center_x], key=lambda p: abs(p[0] - center_x))

# Create a blank mask for the above region
mask_above = np.zeros((img_height, img_width), dtype=np.uint8)

# Define the region above the line between `left_point` and `right_point`
polygon_above = np.array([[0, 0], [img_width, 0], right_point, left_point], dtype=np.int32)

# Draw the polygon on the mask
cv2.fillPoly(mask_above, [polygon_above], 255)

# Apply the mask to the example image
example_img_with_mask_above = cv2.bitwise_and(example_img, example_img, mask=mask_above)

# Plot the mask and example image with the mask applied
fig, axs = plt.subplots(1, 2, figsize=(15, 8))

# Mask for the above region
axs[0].imshow(mask_above, cmap='gray', extent=[0, img_width, img_height, 0])
axs[0].set_title("Mask for Region Above the Line")
axs[0].set_xlabel("X Coordinate (Width)")
axs[0].set_ylabel("Y Coordinate (Height)")

# Example image with mask applied
axs[1].imshow(cv2.cvtColor(example_img_with_mask_above, cv2.COLOR_BGR2RGB))
axs[1].set_title("Example Image with Above Mask Applied")
axs[1].axis('off')

plt.tight_layout()
plt.show()


In [ ]:
# Perform the deduction
deducted_mask_above = cv2.bitwise_and(mask_above, cv2.bitwise_not(mask_approx))

# Plot the masks side by side
fig, axs = plt.subplots(1, 3, figsize=(20, 8))

# Original `another_mask`
axs[0].imshow(mask_above, cmap='gray', extent=[0, img_width, img_height, 0])
axs[0].set_title("Original Mask")
axs[0].set_xlabel("X Coordinate (Width)")
axs[0].set_ylabel("Y Coordinate (Height)")

# `mask_approx`
axs[1].imshow(mask_approx, cmap='gray', extent=[0, img_width, img_height, 0])
axs[1].set_title("Mask to Deduct (mask_approx)")
axs[1].set_xlabel("X Coordinate (Width)")
axs[1].set_ylabel("Y Coordinate (Height)")

# Resulting `deducted_mask`
axs[2].imshow(deducted_mask_above, cmap='gray', extent=[0, img_width, img_height, 0])
axs[2].set_title("Resulting Mask After Deduction")
axs[2].set_xlabel("X Coordinate (Width)")
axs[2].set_ylabel("Y Coordinate (Height)")

plt.tight_layout()
plt.show()

# Save the resulting mask for later use or inspection
output_path = os.path.join(folder, "deducted_mask.png")
cv2.imwrite(output_path, deducted_mask_above)
print(f"Deducted mask saved to {output_path}")


In [ ]:
# Find contours in the mask
contours, _ = cv2.findContours(deducted_mask_above, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Create a blank mask
filled_mask_above = np.zeros_like(deducted_mask_above)

# Fill the contours
cv2.drawContours(filled_mask_above, contours, -1, 255, thickness=cv2.FILLED)

# Plot the original mask and the filled mask side by side
fig, axs = plt.subplots(1, 2, figsize=(15, 8))

# Original mask
axs[0].imshow(deducted_mask_above, cmap='gray', extent=[0, img_width, img_height, 0])
axs[0].set_title("Original Mask with Holes")
axs[0].set_xlabel("X Coordinate (Width)")
axs[0].set_ylabel("Y Coordinate (Height)")

# Filled mask
axs[1].imshow(filled_mask_above, cmap='gray', extent=[0, img_width, img_height, 0])
axs[1].set_title("Mask with Holes Filled")
axs[1].set_xlabel("X Coordinate (Width)")
axs[1].set_ylabel("Y Coordinate (Height)")

plt.tight_layout()
plt.show()

# Save the filled mask
output_path = os.path.join(folder, "filled_mask_above.png")
cv2.imwrite(output_path, filled_mask_above)
print(f"Filled mask saved to {output_path}")


In [ ]:
# Create a blank mask for the below region
mask_below = np.zeros((img_height, img_width), dtype=np.uint8)

# Define the region below the line between `left_point` and `right_point`
polygon_below = np.array([[0, img_height], [img_width, img_height], right_point, left_point], dtype=np.int32)

# Draw the polygon on the mask
cv2.fillPoly(mask_below, [polygon_below], 255)

# Apply the mask to the example image
example_img_with_mask_below = cv2.bitwise_and(example_img, example_img, mask=mask_below)

# Plot the mask and example image with the mask applied
fig, axs = plt.subplots(1, 2, figsize=(15, 8))

# Mask for the below region
axs[0].imshow(mask_below, cmap='gray', extent=[0, img_width, img_height, 0])
axs[0].set_title("Mask for Region Below the Line")
axs[0].set_xlabel("X Coordinate (Width)")
axs[0].set_ylabel("Y Coordinate (Height)")

# Example image with mask applied
axs[1].imshow(cv2.cvtColor(example_img_with_mask_below, cv2.COLOR_BGR2RGB))
axs[1].set_title("Example Image with Below Mask Applied")
axs[1].axis('off')

plt.tight_layout()
plt.show()


In [ ]:
# Perform the deduction
mask_inv = cv2.bitwise_not(mask_below)
deducted_mask_below = cv2.bitwise_or(mask_approx, cv2.bitwise_not(mask_inv))

# Plot the masks side by side
fig, axs = plt.subplots(1, 3, figsize=(20, 8))

# Original `another_mask`
axs[0].imshow(mask_below, cmap='gray', extent=[0, img_width, img_height, 0])
axs[0].set_title("Original Mask")
axs[0].set_xlabel("X Coordinate (Width)")
axs[0].set_ylabel("Y Coordinate (Height)")

# `mask_approx`
axs[1].imshow(mask_approx, cmap='gray', extent=[0, img_width, img_height, 0])
axs[1].set_title("Mask to Deduct (mask_approx)")
axs[1].set_xlabel("X Coordinate (Width)")
axs[1].set_ylabel("Y Coordinate (Height)")

# Resulting `deducted_mask`
axs[2].imshow(deducted_mask_below, cmap='gray', extent=[0, img_width, img_height, 0])
axs[2].set_title("Resulting Mask After Deduction")
axs[2].set_xlabel("X Coordinate (Width)")
axs[2].set_ylabel("Y Coordinate (Height)")

plt.tight_layout()
plt.show()

# Save the resulting mask for later use or inspection
output_path = os.path.join(folder, "deducted_mask_below.png")
cv2.imwrite(output_path, deducted_mask_below)
print(f"Deducted mask saved to {output_path}")


In [ ]:
# Find contours in the mask
contours, _ = cv2.findContours(deducted_mask_below, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Create a blank mask
filled_mask_below = np.zeros_like(deducted_mask_below)

# Fill the contours
cv2.drawContours(filled_mask_below, contours, -1, 255, thickness=cv2.FILLED)

# Plot the original mask and the filled mask side by side
fig, axs = plt.subplots(1, 2, figsize=(15, 8))

# Original mask
axs[0].imshow(deducted_mask_below, cmap='gray', extent=[0, img_width, img_height, 0])
axs[0].set_title("Original Mask with Holes")
axs[0].set_xlabel("X Coordinate (Width)")
axs[0].set_ylabel("Y Coordinate (Height)")

# Filled mask
axs[1].imshow(filled_mask_below, cmap='gray', extent=[0, img_width, img_height, 0])
axs[1].set_title("Mask with Holes Filled")
axs[1].set_xlabel("X Coordinate (Width)")
axs[1].set_ylabel("Y Coordinate (Height)")

plt.tight_layout()
plt.show()

# Save the filled mask
output_path = os.path.join(folder, "filled_mask_below.png")
cv2.imwrite(output_path, filled_mask_below)
print(f"Filled mask saved to {output_path}")


In [ ]:
# Create a new mask by deducting the overlap between mask_above and mask_outside
mask_above_deduction = cv2.bitwise_and(mask_above, cv2.bitwise_not(mask_outside))

# Apply the deducted mask to the example image
example_img_with_mask_above_deduction = cv2.bitwise_and(example_img, example_img, mask=mask_above_deduction)

# Plot the original mask_above, mask_outside, and the result of the deduction
fig, axs = plt.subplots(1, 3, figsize=(18, 6))

# Original mask_above
axs[0].imshow(mask_above, cmap='gray', extent=[0, img_width, img_height, 0])
axs[0].set_title("Original Mask Above")
axs[0].set_xlabel("X Coordinate (Width)")
axs[0].set_ylabel("Y Coordinate (Height)")

# mask_outside (triangular areas)
axs[1].imshow(mask_outside, cmap='gray', extent=[0, img_width, img_height, 0])
axs[1].set_title("Mask Outside (Triangular Areas)")
axs[1].set_xlabel("X Coordinate (Width)")

# Resulting mask_above_deduction
axs[2].imshow(mask_above_deduction, cmap='gray', extent=[0, img_width, img_height, 0])
axs[2].set_title("Mask Above with Outside Deducted")
axs[2].set_xlabel("X Coordinate (Width)")

plt.tight_layout()
plt.show()

# Plot the example image with the deducted mask applied
plt.figure(figsize=(10, 8))
plt.imshow(cv2.cvtColor(example_img_with_mask_above_deduction, cv2.COLOR_BGR2RGB))
plt.title("Example Image with Deducted Mask Above Applied")
plt.axis('off')
plt.show()


In [ ]:
mask_inside = cv2.bitwise_not(deducted_mask_outside)

plt.figure(figsize=(10, 8))
plt.imshow(cv2.cvtColor(mask_inside, cv2.COLOR_BGR2RGB))
plt.title("Example Image with Deducted Mask Above Applied")
plt.axis('off')
plt.show()

In [ ]:
masks = {
    'filled_mask_below': filled_mask_below,
    'filled_mask_above': filled_mask_above,
    'deducted_mask_outside': deducted_mask_outside,
    'mask_inside': mask_inside
}


output_folder = r'D:\PAIND\DATA\20240527_Data_Prozess_01\webcam\batch\output_grayscale_gaussian'

for mask_name, mask in masks.items():
    # Save as PNG image
    mask_filename = os.path.join(output_folder, f"{mask_name}.png")
    cv2.imwrite(mask_filename, mask)
    print(f"Mask '{mask_name}' saved to {mask_filename}")


In [ ]:
import cv2
import os
from tqdm import tqdm

# Paths and parameters
image_folder = r''
output_folder = r'D:\PAIND\DATA\20240527_Data_Prozess_01\webcam\batch\output_grayscale_gaussian'

# Masks dictionary containing mask names and paths
masks = {
    "mask_above": os.path.join(output_folder, "mask_above.png"),
    "mask_below": os.path.join(output_folder, "mask_below.png"),
    "mask_outside": os.path.join(output_folder, "mask_outside.png"),
    "mask_above_deduction": os.path.join(output_folder, "mask_above_deduction.png"),
    "mask_approx": os.path.join(output_folder, "mask_approx.png"),
}

# Get sorted list of images to process
images_to_process = sorted([img for img in os.listdir(image_folder) if img.endswith(".jpg")])

# Iterate over each mask dynamically
for mask_name, mask_path in tqdm(masks.items(), desc="Processing Masks"):
    # Load the mask
    if not os.path.exists(mask_path):
        print(f"Mask not found: {mask_path}")
        continue
    
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    
    # Create output folder for each mask
    mask_output_folder = os.path.join(output_folder, f"masked_images_{mask_name}")
    os.makedirs(mask_output_folder, exist_ok=True)
    
    # Apply the mask to all images
    for image_name in tqdm(images_to_process, desc=f"Applying {mask_name}"):
        image_path = os.path.join(image_folder, image_name)
        image = cv2.imread(image_path)
        
        # Resize the mask if necessary
        if mask.shape != image.shape[:2]:
            mask_resized = cv2.resize(mask, (image.shape[1], image.shape[0]))
        else:
            mask_resized = mask
        
        # Apply the mask to the image
        masked_image = cv2.bitwise_and(image, image, mask=mask_resized)
        
        # Save the masked image
        output_image_path = os.path.join(mask_output_folder, f"masked_{image_name}")
        cv2.imwrite(output_image_path, masked_image)
    
    print(f"Finished applying {mask_name}. Results saved to {mask_output_folder}")
